# Finding similar neighborhoods in different cities

In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json

from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

import folium

from bs4 import BeautifulSoup as bsoup

In [2]:
# Foursquare credentials
CLIENT_ID = '' # Foursquare client id
CLIENT_SECRET = '' # Foursquare client secret
VERSION = '20210606'
LIMIT = 100

In [16]:
# Geolocator object
geolocator = Nominatim(user_agent="my_request")

### Baner location

In [17]:
baner_loc = geolocator.geocode("Baner, Pune, IN")
baner_lat = baner_loc.latitude
baner_long = baner_loc.longitude

print("Geographical co-ordinates of Bangalore: ", baner_lat, ",", baner_long)

Geographical co-ordinates of Bangalore:  18.5642431 , 73.7768573


### Bangalore location and neighborhoods

In [5]:
# Bangalore location
bngr_loc = geolocator.geocode("Bangalore, IN")
bngr_lat = bngr_loc.latitude
bngr_long = bngr_loc.longitude
print("Bangalore co-ordinates: ", bngr_lat, ",", bngr_long)

Bangalore co-ordinates:  12.9791198 , 77.5912997


In [6]:
# List of Bangalore neighborhoods
bng_wiki = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore")
bng_soup = bsoup(bng_wiki.text, "html.parser")
bngr_nbrh = pd.DataFrame(columns = ["Suburb", "Neighborhood"])

table_list = pd.read_html(bng_wiki.text)[0:8]
for i in range(8):
    table = table_list[i]
    
    for name in table["Name"].values:
        nbr = name + ", Bangalore Urban, IN"
        bngr_nbrh = bngr_nbrh.append({"Suburb": bng_soup.find_all("h2")[i+1].span.string,
                         "Neighborhood": nbr}, ignore_index= True)
        
print(bngr_nbrh.shape)


(65, 2)


In [7]:
# Changing some neighborhood names too avoid errors in geolocator
bngr_nbrh["Neighborhood"][0] = "Bangalore Cantonment, Bangalore Urban, IN"
bngr_nbrh["Neighborhood"][3] = "Jeevan Bhima Nagar, Bangalore Urban, IN"
bngr_nbrh["Neighborhood"][5] = "Bengaluru Market, Bangalore Urban, IN"
bngr_nbrh["Neighborhood"][12] = "C. V. Raman Nagar, Bangalore Urban, IN"

In [18]:
# Getting co-ordinates of neighborhoods
bngr_nbrh_df = pd.DataFrame(columns = ["Suburb", "Neighborhood", "Latitude", "Longitude"])

for sub, nbrh in zip(bngr_nbrh["Suburb"], bngr_nbrh["Neighborhood"]):

    nbrh_loc = geolocator.geocode(nbrh)
    nbrh_lat = nbrh_loc.latitude
    nbrh_lon = nbrh_loc.longitude

    bngr_nbrh_df = bngr_nbrh_df.append({"Suburb": sub,
                                        "Neighborhood": nbrh.split(",")[0],
                                       "Latitude":nbrh_lat,
                                       "Longitude":nbrh_lon}, ignore_index=True)

bngr_nbrh_df.head()

,Suburb,Neighborhood,Latitude,Longitude
0,Central,Bangalore Cantonment,12.993687,77.598000
1,Central,Domlur,12.962467,77.638196
2,Central,Indiranagar,12.973291,77.640467
3,Central,Jeevan Bhima Nagar,12.967807,77.656837
4,Central,Malleswaram,13.002735,77.570325


In [61]:
bngr_nbrh_df["City"] = ["Bangalore" for i in range(bngr_nbrh_df.shape[0])]

In [100]:
# Bangalore neighborhoods map
bngr_map = folium.Map(location = [bngr_lat, bngr_long], zoom_start=11, max_zoom = 11, min_zoom = 11)

for nbr, lt, lng in zip(bngr_nbrh_df["Neighborhood"], bngr_nbrh_df["Latitude"], bngr_nbrh_df["Longitude"]):
    label = nbr+ ","+str(lt)+","+str(lng)
    folium.CircleMarker(
    [lt, lng],
    radius = 5,
    color = "blue",
    fill = True,
    fill_color = "blue",
    fill_opacity = 0.6,
    popup = label).add_to(bngr_map)
    
bngr_map

### Delhi location and neighborhoods

In [20]:
# Delhi location
delhi_loc = geolocator.geocode("Delhi, IN")
delhi_lat = delhi_loc.latitude
delhi_long = delhi_loc.longitude

print("Dlhi co-ordinates:", delhi_lat, ",", delhi_long)

Dlhi co-ordinates: 28.6517178 , 77.2219388


In [21]:
# Delhi neighborhoods wiki page
delhi_get = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_of_Delhi")
delhi_page = bsoup(delhi_get.text, "html.parser")

In [22]:
# Creating Delhi neighborhood dataframe
delhi_nbr_df = pd.DataFrame(columns = ["Suburb", "Neighborhood", "Latitude", "Longitude"])

for a in range(1, 11):
    sub = delhi_page.find_all("h2")[a].span.string
    for item in delhi_page.find_all("ul")[a].find_all("li"):
        try:
            nbr = item.string
            nbr_loc = geolocator.geocode(nbr + ", Delhi, IN")
            nbr_lat = nbr_loc.latitude
            nbr_long = nbr_loc.longitude
            
            delhi_nbr_df = delhi_nbr_df.append({"Suburb": sub,
                                               "Neighborhood": nbr,
                                               "Latitude": nbr_lat,
                                               "Longitude": nbr_long}, ignore_index=True)
        
        except:
            continue

In [30]:
delhi_nbr_df.at[34, "Neighborhood"] = "Rajendra Nagar"
delhi_nbr_df.at[34, "Latitude"] = geolocator.geocode("Rajendra Nagar, Delhi, IN").latitude
delhi_nbr_df.at[34, "Longitude"] = geolocator.geocode("Rajendra Nagar, Delhi, IN").longitude

In [59]:
delhi_nbr_df["City"] = ["Delhi" for i in range(delhi_nbr_df.shape[0])]

In [60]:
delhi_nbr_df.head()

,Suburb,Neighborhood,Latitude,Longitude,City
0,North West Delhi,Adarsh Nagar,28.716580,77.170422,Delhi
1,North West Delhi,Ashok Vihar,28.699453,77.184826,Delhi
2,North West Delhi,Begum Pur,28.725503,77.058371,Delhi
3,North West Delhi,Karala,28.735140,77.032511,Delhi
4,North West Delhi,Narela,28.842610,77.091835,Delhi


In [101]:
# Delhi neighborhood map
delhi_map = folium.Map(location = [delhi_lat, delhi_long], zoom_start=10, max_zoom = 10, min_zoom = 10)

for nbr, lt, lng in zip(delhi_nbr_df["Neighborhood"], delhi_nbr_df["Latitude"], delhi_nbr_df["Longitude"]):
    label = nbr + " " + str(lt) + " " + str(lng)
    folium.CircleMarker(
    [lt, lng],
    radius = 5,
    color = "blue",
    popup = label,
    fill = True,
    fill_color = "blue",
    fill_opacity = 0.5).add_to(delhi_map)
    
delhi_map

In [64]:
combined_nbrh = pd.DataFrame({"City": ["Pune"],
                             "Suburb": ["Baner"],
                             "Neighborhood": ["Baner"],
                             "Latitude": [baner_lat],
                             "Longitude": [baner_long]})
combined_nbrh = combined_nbrh.append(bngr_nbrh_df).append(delhi_nbr_df).reset_index(drop = True)
combined_nbrh.head()

,City,Suburb,Neighborhood,Latitude,Longitude
0,Pune,Baner,Baner,18.564243,73.776857
1,Bangalore,Central,Bangalore Cantonment,12.993687,77.598000
2,Bangalore,Central,Domlur,12.962467,77.638196
3,Bangalore,Central,Indiranagar,12.973291,77.640467
4,Bangalore,Central,Jeevan Bhima Nagar,12.967807,77.656837


In [67]:
combined_nbrh_sorted = combined_nbrh.sort_values("Neighborhood")

### Getting venues in different neighborhoods

In [32]:
# Function to get categories of venues
def get_category(row):
    categories_list = row['venue.categories']
    
    return categories_list[0]['name']

In [33]:
# Function to get venues in a list neighborhoods
def getNearbyVenues(city, suburbs, names, latitudes, longitudes, radius = 750):
    
    venues_list = []
    for name, suburb, lat, lng in zip(names, suburbs, latitudes, longitudes):
        
        
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,
        LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
        city,
        suburb,
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                             'Suburb',
                             'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

#### Baner venues

In [34]:
# Getting Baner venues from foursquare
radius = 750
baner_url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        baner_lat,
        baner_long,
        radius,
        LIMIT)

baner_results = requests.get(baner_url).json()['response']['groups'][0]['items']

In [35]:
# Baner venues dataframe
baner_df = json_normalize(baner_results)
baner_venue_no = baner_df.shape[0]
print(baner_venue_no)
baner_df.head(2)

23


<ipython-input-35-b533e683e91c>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  baner_df = json_normalize(baner_results)


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.location.postalCode
0,e-0-56e438f6498e59585abc9507-0,0,"[{'summary': 'This spot is popular', 'type': '...",56e438f6498e59585abc9507,Urban Foundry,Balewadi high Street,18.569289,73.774725,"[{'label': 'display', 'lat': 18.56928940966670...",605,IN,Pune,Mahārāshtra,India,"[Balewadi high Street, Pune, Mahārāshtra, India]","[{'id': '4bf58dd8d48988d121941735', 'name': 'L...",0,[],NaN,NaN
1,e-0-571cdb9b498e2192c426cb14-1,0,"[{'summary': 'This spot is popular', 'type': '...",571cdb9b498e2192c426cb14,Terttulia,Balewadi High Street,18.569280,73.774695,"[{'label': 'display', 'lat': 18.56927994563982...",605,IN,Puṇe,Mahārāshtra,India,"[Balewadi High Street, Puṇe, Mahārāshtra, India]","[{'id': '52e81612bcbc57f1066b79f1', 'name': 'B...",0,[],NaN,NaN


In [36]:
# Cleaning Baner venues data frame to contain only needed columns
baner_venues = pd.DataFrame({"City": ["Pune" for i in range(baner_venue_no)],
                             "Suburb": ["Baner" for i in range(baner_venue_no)],
                            "Neighborhood":["Baner" for i in range(baner_venue_no)],
                            "Neighborhood Latitude": [baner_lat for i in range(baner_venue_no)],
                            "Neighborhood Longitude": [baner_long for i in range(baner_venue_no)],
                            "Venue": baner_df["venue.name"],
                            "Venue Latitude": baner_df["venue.location.lat"],
                            "Venue Longitude": baner_df["venue.location.lng"],
                            "Venue Category": baner_df.apply(get_category, axis = 1)})

In [37]:
baner_venues.head(2)

,City,Suburb,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Pune,Baner,Baner,18.564243,73.776857,Urban Foundry,18.569289,73.774725,Lounge
1,Pune,Baner,Baner,18.564243,73.776857,Terttulia,18.569280,73.774695,Bistro


#### Bangalore venues

In [38]:
# Nearby venues in Bangalore neighborhoods
bngr_venues = getNearbyVenues(city = "Bangalore",
                              suburbs = bngr_nbrh_df["Suburb"],
                              names = bngr_nbrh_df["Neighborhood"],
                             latitudes = bngr_nbrh_df["Latitude"],
                             longitudes = bngr_nbrh_df["Longitude"],
                             radius = 750)
print(bngr_venues.shape)

(1287, 9)


In [39]:
# Nearby venues in Delhi neighborhoods
delhi_venues = getNearbyVenues(city = "Delhi",
                               suburbs = delhi_nbr_df["Suburb"],
                              names = delhi_nbr_df["Neighborhood"],
                              latitudes = delhi_nbr_df["Latitude"],
                              longitudes = delhi_nbr_df["Longitude"],
                              radius = 750)

print(delhi_venues.shape)

(1331, 9)


In [40]:
bngr_venues.sample(5)

,City,Suburb,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
754,Bangalore,South-Eastern,HSR Layout,12.911623,77.638862,Agara Lake Trail,12.916410,77.638941,Trail
940,Bangalore,Southern,Banashankari,12.915219,77.573621,Ande ka funda,12.911295,77.578370,Fast Food Restaurant
175,Bangalore,Central,Jeevan Bhima Nagar,12.967807,77.656837,Thippasandra 6th Cross Park,12.970819,77.650808,Park
1009,Bangalore,Southern,J. P. Nagar,12.909694,77.586607,Dilli Chat Walla,12.906327,77.588196,Fast Food Restaurant
926,Bangalore,South-Eastern,Madiwala,12.921227,77.617951,Star Mess,12.923639,77.612763,Indian Restaurant


In [41]:
delhi_venues.sample(5)

,City,Suburb,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1045,Delhi,South East Delhi,Govindpuri,28.535156,77.263794,Fitness Finesse,28.537289,77.256761,Gym
380,Delhi,New Delhi,Connaught Place,28.631402,77.219379,Domino's Pizza,28.633301,77.222796,Pizza Place
964,Delhi,South East Delhi,New Friends Colony,28.567101,77.269764,Domino's Pizza,28.562000,77.270000,Pizza Place
805,Delhi,South East Delhi,Lodhi Colony,28.586203,77.226858,Elma's Brasserie,28.584493,77.226647,Restaurant
10,Delhi,North West Delhi,Ashok Vihar,28.699453,77.184826,Chatrasaal Stadium,28.704008,77.187431,Track Stadium


In [42]:
combined_venues = baner_venues.append(bngr_venues).append(delhi_venues)
combined_venues.shape

(2641, 9)

In [55]:
combined_venues.head(8)

,City,Suburb,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Pune,Baner,Baner,18.564243,73.776857,Urban Foundry,18.569289,73.774725,Lounge
1,Pune,Baner,Baner,18.564243,73.776857,Terttulia,18.569280,73.774695,Bistro
2,Pune,Baner,Baner,18.564243,73.776857,Incognito,18.570376,73.774607,Italian Restaurant
3,Pune,Baner,Baner,18.564243,73.776857,Cafe Coffee Day,18.564363,73.779084,Café
4,Pune,Baner,Baner,18.564243,73.776857,Way Down South,18.566366,73.770707,South Indian Restaurant
5,Pune,Baner,Baner,18.564243,73.776857,Zikomo,18.564132,73.780005,Indian Restaurant
6,Pune,Baner,Baner,18.564243,73.776857,KFC,18.564908,73.774485,Fast Food Restaurant
7,Pune,Baner,Baner,18.564243,73.776857,McDonald's,18.570421,73.774619,Fast Food Restaurant


In [45]:
# Unique venue categories and count in the combined dataframe
combined_venues["Venue Category"].value_counts()

Indian Restaurant                  382
Café                               173
Fast Food Restaurant               123
Coffee Shop                        108
Hotel                              101
Pizza Place                         89
Chinese Restaurant                  79
Restaurant                          64
Bakery                              50
Ice Cream Shop                      40
Park                                40
Sandwich Place                      38
Department Store                    38
Asian Restaurant                    38
Italian Restaurant                  35
Dessert Shop                        34
Bar                                 34
Lounge                              31
Snack Place                         31
Market                              29
Vegetarian / Vegan Restaurant       29
Clothing Store                      28
Breakfast Spot                      25
Shopping Mall                       25
Light Rail Station                  24
BBQ Joint                

For better reliability of the k-means clustering method, we chose the categories that have count greater than equal to 10.

In [46]:
# Venues dataframe with filtered categories
filter_categories = combined_venues["Venue Category"].value_counts().index[combined_venues["Venue Category"].value_counts().values >= 10]

combined_venues_filtered = combined_venues[combined_venues["Venue Category"].isin(filter_categories)].reset_index(drop = True)
print(combined_venues_filtered.shape)
combined_venues_filtered.head()

(2064, 9)


,City,Suburb,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Pune,Baner,Baner,18.564243,73.776857,Urban Foundry,18.569289,73.774725,Lounge
1,Pune,Baner,Baner,18.564243,73.776857,Incognito,18.570376,73.774607,Italian Restaurant
2,Pune,Baner,Baner,18.564243,73.776857,Cafe Coffee Day,18.564363,73.779084,Café
3,Pune,Baner,Baner,18.564243,73.776857,Way Down South,18.566366,73.770707,South Indian Restaurant
4,Pune,Baner,Baner,18.564243,73.776857,Zikomo,18.564132,73.780005,Indian Restaurant


In [47]:
# Convert dataframe to perform k-means on venue categories
venues_onehot = pd.get_dummies(combined_venues_filtered[["Venue Category"]], prefix = "", prefix_sep = "")


In [48]:
# Add necessary location columns to onehot dataframe
venues_onehot["Neighborhood"] = combined_venues_filtered["Neighborhood"]
venues_onehot["Suburb"] = combined_venues_filtered["Suburb"]
venues_onehot["City"] = combined_venues_filtered["City"]

In [50]:
# Reorder dataframe columns
columns = [venues_onehot.columns[-1]] + [venues_onehot.columns[-2]] + [venues_onehot.columns[-3]] + list(venues_onehot.columns[:-3])
venues_onehot_filtered = venues_onehot[columns]

In [74]:
# Group by neighborhood and take mean of frequency of venues
venues_onehot_grouped = venues_onehot_filtered.groupby("Neighborhood").mean().reset_index()

In [81]:
# Add locations to grouped dataframe
venues_onehot_final = venues_onehot_grouped.merge(combined_nbrh_sorted[["City", "Suburb", "Neighborhood"]], on = "Neighborhood", how = "left")
final_columns = [venues_onehot_final.columns[-2]] + [venues_onehot_final.columns[-1]] + list(venues_onehot_final.columns[:-2])
venues_onehot_final = venues_onehot_final[final_columns]
venues_onehot_final.head()

,City,Suburb,Neighborhood,American Restaurant,Arcade,Asian Restaurant,BBQ Joint,Bakery,Bar,Breakfast Spot,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Department Store,Dessert Shop,Diner,Donut Shop,Fast Food Restaurant,Flea Market,Food Court,Food Truck,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Light Rail Station,Lounge,Market,Metro Station,Movie Theater,Multiplex,Nightclub,Park,Pizza Place,Playground,Plaza,Pub,Restaurant,Sandwich Place,Shopping Mall,Snack Place,South Indian Restaurant,Steakhouse,Tea Room,Theater,Train Station,Vegetarian / Vegan Restaurant
0,Delhi,North West Delhi,Adarsh Nagar,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.250000,0.0,0.0,0.250000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.25,0.0
1,Delhi,South East Delhi,Alaknanda,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.25,0.0,0.000000,0.000000,0.0,0.25,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
2,Delhi,East Delhi,Anand Vihar,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.000000,0.0,0.0,0.142857,0.0,0.00,0.0,0.142857,0.142857,0.0,0.00,0.142857,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
3,Bangalore,Southern suburbs,Arekere,0.0,0.0,0.0,0.0,0.086957,0.043478,0.043478,0.000000,0.043478,0.086957,0.000000,0.043478,0.0,0.086957,0.0,0.0,0.0,0.043478,0.0,0.000000,0.043478,0.00,0.043478,0.000000,0.086957,0.217391,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.0,0.00,0.086957,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.043478,0.0,0.0,0.0,0.00,0.0
4,Delhi,West Delhi,Ashok Nagar,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.111111,0.000000,0.0,0.000000,0.0,0.0,0.0,0.222222,0.0,0.111111,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.0,0.0,0.111111,0.0,0.00,0.0,0.000000,0.111111,0.0,0.00,0.111111,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0


In [84]:
# Filtered data for clustering
venues_clustering = venues_onehot_final.drop(["City", "Suburb", "Neighborhood"], axis = 1)

In [86]:
# K-Means clustering
kclusters = 5

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_clustering)
kmeans.labels_[0:10]

array([0, 2, 1, 0, 4, 4, 0, 4, 4, 0])

In [87]:
# Add cluster labels to onehot dataframe
venues_onehot_final["Cluster Label"] = kmeans.labels_

In [88]:
# Cluster label of Baner, Pune
venues_onehot_final[venues_onehot_final["Neighborhood"] == "Baner"]["Cluster Label"]

11    0
Name: Cluster Label, dtype: int32

In [91]:
# Neighborhoods that have same cluster label as Baner, Pune
baner_cluster_nbrh = venues_onehot_final[venues_onehot_final["Cluster Label"] == 0][["City", "Suburb", "Neighborhood"]]

In [94]:
# Neighborhoods in Bangalore similar to Baner
baner_cluster_nbrh[baner_cluster_nbrh["City"] == "Bangalore"]

,City,Suburb,Neighborhood
3,Bangalore,Southern suburbs,Arekere
6,Bangalore,South-Eastern,BTM Layout
9,Bangalore,Southern,Banashankari
10,Bangalore,North-Eastern,Banaswadi
12,Bangalore,Central,Bangalore Cantonment
14,Bangalore,Southern,Basavanagudi
18,Bangalore,Central,Bengaluru Market
19,Bangalore,South-Eastern,Bommanahalli
20,Bangalore,South-Eastern,Bommasandra
21,Bangalore,Eastern,C. V. Raman Nagar


In [97]:
# Neighborhoods in Delhi similar to Baner
baner_cluster_nbrh[baner_cluster_nbrh["City"] == "Delhi"]

,City,Suburb,Neighborhood
0,Delhi,North West Delhi,Adarsh Nagar
13,Delhi,New Delhi,Barakhamba Road
22,Delhi,New Delhi,Chanakyapuri
23,Delhi,Central Delhi,Chandni Chowk
24,Delhi,South Delhi,Chattarpur
25,Delhi,South East Delhi,Chittaranjan Park
27,Delhi,New Delhi,Connaught Place
29,Delhi,Central Delhi,Daryaganj
30,Delhi,South East Delhi,Defence Colony
35,Delhi,East Delhi,East Vinod Nagar
